In [63]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vocabulary, Vectorizer, HeadQA, HeadQA_IR, clean_words, parse_dataset, parse_ir_dataset, random_oversamplig
from training import train, validate, evaluate

import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


In [65]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [57]:
training[0]

{'answers': [{'aid': 1, 'atext': 'Son de tipo todo o nada.'},
  {'aid': 2, 'atext': 'Son hiperpolarizantes.'},
  {'aid': 3, 'atext': 'Se pueden sumar.'},
  {'aid': 4, 'atext': 'Se propagan a largas distancias.'},
  {'aid': 5, 'atext': 'Presentan un periodo refractario.'}],
 'category': 'biology',
 'image': '',
 'name': 'Cuaderno_2013_1_B',
 'qid': 1,
 'qtext': 'Los potenciales postsinápticos excitadores:',
 'ra': 3,
 'year': '2013'}

In [5]:
training_instances = parse_ir_dataset(training)
validation_instances = parse_ir_dataset(validation)
testing_instances = parse_ir_dataset(testing)

oversampled_training = random_oversamplig(training_instances)

In [51]:
oversampled_training[0]

{'question': 'Los potenciales postsinápticos excitadores:',
 'answer': 'Son de tipo todo o nada.',
 'tok_qtext': ['Los', 'potenciales', 'postsinápticos', 'excitadores', ':'],
 'tok_atext': ['Son', 'de', 'tipo', 'todo', 'o', 'nada', '.'],
 'label': 0,
 'category': 'biology'}

In [66]:
vectorizer = Vectorizer.vectorize_ir_dataset(oversampled_training)

In [67]:
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [68]:
trainset = HeadQA_IR(instances=training_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
validset = HeadQA_IR(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
testset = HeadQA_IR(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=15)

In [69]:
trainset.instances[0]

{'question': 'Los potenciales postsinápticos excitadores:',
 'answer': 'Son de tipo todo o nada.',
 'tok_qtext': ['Los', 'potenciales', 'postsinápticos', 'excitadores', ':'],
 'tok_atext': ['Son', 'de', 'tipo', 'todo', 'o', 'nada', '.'],
 'label': 0,
 'category': 'biology'}

In [70]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [ ]:
class LSTM_QA(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim, x_size, n_classes, embedding_dim=300): 
        super(BasicLSTM, self).__init__()
        
    def forward(self, x_0, x_1):
        pass
    